![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_CamemBertForQuestionAnswering.ipynb)

## Import ONNX CamemBertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `CamemBertForQuestionAnswering` is only available since in `Spark NLP 5.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for question answering via `CamembertForQuestionAnswering` or `TFCamembertForQuestionAnswering`. These models are usually under `Question Answering` category and have `camembert` in their labels
- Reference: [TFCamembertForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=question-answering&sort=downloads)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- CamembertTokenizer requires the `SentencePiece` library, so we install that as well

In [3]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum sentencepiece tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/49

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [illuin/camembert-base-fquad](https://huggingface.co/illuin/camembert-base-fquad) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.

In [4]:
from optimum.onnxruntime import ORTModelForQuestionAnswering
import tensorflow as tf

MODEL_NAME = 'illuin/camembert-base-fquad'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

(…)bert-base-fquad/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Framework not specified. Using pt to export to ONNX.


pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

(…)fquad/resolve/main/tokenizer_config.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

(…)uad/resolve/main/sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

(…)uad/resolve/main/special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {ONNX_MODEL}

total 433264
-rw-r--r-- 1 root root       717 Nov  3 13:33 config.json
-rw-r--r-- 1 root root 440413011 Nov  3 13:33 model.onnx
-rw-r--r-- 1 root root    810912 Nov  3 13:33 sentencepiece.bpe.model
-rw-r--r-- 1 root root       241 Nov  3 13:33 special_tokens_map.json
-rw-r--r-- 1 root root       569 Nov  3 13:33 tokenizer_config.json
-rw-r--r-- 1 root root   2418877 Nov  3 13:33 tokenizer.json


- As you can see, we need to move `sentencepiece.bpe.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [6]:
!mkdir {ONNX_MODEL}/assets

In [7]:
!mv {ONNX_MODEL}/sentencepiece.bpe.model {ONNX_MODEL}/assets

Voila! We have our `spiece.model` inside assets directory

In [8]:
!ls -lR {ONNX_MODEL}

onnx_models/illuin/camembert-base-fquad:
total 432476
drwxr-xr-x 2 root root      4096 Nov  3 13:33 assets
-rw-r--r-- 1 root root       717 Nov  3 13:33 config.json
-rw-r--r-- 1 root root 440413011 Nov  3 13:33 model.onnx
-rw-r--r-- 1 root root       241 Nov  3 13:33 special_tokens_map.json
-rw-r--r-- 1 root root       569 Nov  3 13:33 tokenizer_config.json
-rw-r--r-- 1 root root   2418877 Nov  3 13:33 tokenizer.json

onnx_models/illuin/camembert-base-fquad/assets:
total 792
-rw-r--r-- 1 root root 810912 Nov  3 13:33 sentencepiece.bpe.model


## Import and Save CamemBertForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [9]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.4
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 22.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [10]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `CamemBertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [12]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = CamemBertForQuestionAnswering.loadSavedModel(
     f"{ONNX_MODEL}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [13]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [14]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your CamemBertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [15]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 430960
-rw-r--r-- 1 root root 440480388 Nov  3 13:41 camembert_classification_onnx
-rw-r--r-- 1 root root    810912 Nov  3 13:41 camembert_spp
drwxr-xr-x 2 root root      4096 Nov  3 13:41 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [23]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = CamemBertForQuestionAnswering.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

context = "Mon nom est Wolfgang et je vis à Berlin"
question = "Où est-ce que je vis?"

example = spark.createDataFrame([[question, contexilluin/camembert-base-fquadt]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("question", "answer.result").show(truncate=False)

+---------------------+--------+
|question             |result  |
+---------------------+--------+
|Où est-ce que je vis?|[berlin]|
+---------------------+--------+



That's it! You can now go wild and use hundreds of `CamemBertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
